In [ ]:
!pip install gcsfs

In [6]:
import pandas as pd
import gcsfs

In [7]:
fs = gcsfs.GCSFileSystem(project='omina-gcp-resource')
with fs.open('omina-test-set/occupancy-data/occupancy_data.csv') as f:
    df = pd.read_csv(f)

In [22]:
import google.datalab.storage as storage
import pandas as pd
from io import BytesIO

mybucket = storage.Bucket('omina-test-set')
data_csv = mybucket.object('occupancy-data/occupancy_data.csv')

uri = data_csv.uri
%gcs read --object $uri --variable data

df = pd.read_csv(BytesIO(data))
df.head()

,Unnamed: 0,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,1,2015-02-11 14:48:00,21.7600,31.133333,437.333333,1029.666667,0.005021,1
1,2,2015-02-11 14:49:00,21.7900,31.000000,437.333333,1000.000000,0.005009,1
2,3,2015-02-11 14:50:00,21.7675,31.122500,434.000000,1003.750000,0.005022,1
3,4,2015-02-11 14:51:00,21.7675,31.122500,439.000000,1009.500000,0.005022,1
4,5,2015-02-11 14:51:59,21.7900,31.133333,437.333333,1005.666667,0.005030,1


In [23]:
df['date'] = pd.to_datetime(df['date'])
df.index = df['date']
del df['date']
del df['Unnamed: 0']

In [24]:
df = df.sort_values(by='date')
df_group = df.groupby(pd.Grouper(key = 'date', freq = 'D'))
df['NSM'] = df.date.apply(lambda x: x - x.replace(hour=0, minute=0, second=0)).dt.total_seconds()
df['WS'] = ((pd.DatetimeIndex(df.index).dayofweek) // 5 == 1).astype(int)
df.head()

KeyError: 'date'

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [ ]:
df['2015-02-03' : '2015-02-04'].plot(subplots=True, figsize=(15,8), linewidth=5, fontsize=20)
plt.legend(loc='best')

In [18]:
corr = df.corr()

In [ ]:
sns.heatmap(corr, annot="True", cmap="YlGnBu")

In [ ]:
sns.pairplot(df)

In [176]:
y = df["Occupancy"]
X = df.drop("Occupancy", axis=1)

In [177]:
df.isnull().values.any()

False

In [184]:
# df['Length'] = df['date'].apply(lambda x: len(x))
len(df.index)

20560

In [191]:
#Cross validation on the given data to split the training and test set 
from sklearn import cross_validation
from sklearn.cross_validation import train_test_split
#70% of the total data is used for training
train_data_length = int(round(len(df.index)*.07))
print(train_data_length)
X_train,X_test,y_train,y_test = cross_validation.train_test_split(X, y, train_size= train_data_length) 

1439


In [193]:
# Keeping the test data in a dataframe
test_data = df[1439:]